In [ ]:
import numpy as np
import pandas as pd
import itertools
from keras import layers, models
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
#from keras.applications.imagenet_utils import preprocess_input
#import pydot
from sklearn import metrics
from sklearn.metrics import confusion_matrix

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
import os
import datetime
import math
from keras import regularizers
import tensorflow as tf

tf.keras.backend.clear_session()

import multiprocessing as mp
mp.set_start_method('spawn', force=True)

modelToLoad = "InceptionV3"

if modelToLoad == "VGG":
    from keras.applications.vgg19 import VGG19
    from keras.applications.vgg19 import preprocess_input
elif modelToLoad == "ResNet50":
    from keras.applications import ResNet50
    from keras.applications.ResNet50 import preprocess_input
elif modelToLoad == "InceptionV3":
    from keras.applications import InceptionV3
    from keras.applications.inception_v3 import preprocess_input
else:
    print("error")


%matplotlib inline

#to restart GPU to clear memory, run this and then restart kernel
from numba import cuda

tf.keras.backend.clear_session()

from collections import Counter

# cuda.select_device(0)
# cuda.close()
# #to limit tf's memory usage
# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=12288)])


print("loaded")

In [ ]:
if modelToLoad == "VGG":
    image_size_h = 224 #382
    image_size_w = 224 #512
elif modelToLoad == "InceptionV3":
    image_size_h = 299 #382
    image_size_w = 299 #512
else:
    print("using MEAN dimensions of dataset")
    image_size_h = 382
    image_size_w = 512
    
train_batch_size = 32
dev_batch_size = 32
test_batch_size = 32
epochs = 10

includePreTrained = True #include pretrained model in the final model for training
setToUse = "full"

if setToUse == "mini":    
    train_path = "C:\\Users\\Sean\\Downloads\\cs230_jupyter\\VMMRdb_mini\\train"
    dev_path = 'C:\\Users\\Sean\\Downloads\\cs230_jupyter\\VMMRdb_mini\\dev'
    test_path = 'C:\\Users\\Sean\\Downloads\\cs230_jupyter\\VMMRdb_mini\\test'
    bottleneck_file_root = "D:\\Debian\\cs_230_proj\\bottleneck\\inceptionv3\\"
    #"D:\\Debian\\cs_230_proj\\bottleneck\\"

elif setToUse == "full":    
    train_path = "D:/Debian/cs_230_proj/VMMRdb_year2000+_sample100+/train"
    dev_path = "D:/Debian/cs_230_proj/VMMRdb_year2000+_sample100+/dev"
    test_path = "D:/Debian/cs_230_proj/VMMRdb_year2000+_sample100+/test"
    bottleneck_file_root = "D:/Debian/cs_230_proj/VMMRdb_year2000+_sample100+/inceptionv3_fulldata/"

elif setToUse == "mini_new":    
    train_path = "D:/Debian/cs_230_proj/VMMRdb_mini_new/train"
    dev_path = "D:/Debian/cs_230_proj/VMMRdb_mini_new/dev"
    test_path = "D:/Debian/cs_230_proj/VMMRdb_mini_new/test"
    bottleneck_file_root = "D:/Debian/cs_230_proj/VMMRdb_mini_new/inceptionv3_fulldata/"    

top_model_weights_path = "D:/Debian/cs_230_proj/inceptionv3_model.h5"

# datagen_train = ImageDataGenerator(rescale=1. / 255) 
# datagen_dev = ImageDataGenerator(rescale=1. / 255) 
# datagen_test = ImageDataGenerator(rescale=1. / 255) 

# datagen_train = ImageDataGenerator(preprocessing_function = preprocess_input) 
datagen_dev = ImageDataGenerator(preprocessing_function = preprocess_input) 
datagen_test = ImageDataGenerator(preprocessing_function = preprocess_input)

datagen_train = ImageDataGenerator(preprocessing_function = preprocess_input,
                                    rotation_range=8, 
                                    width_shift_range=0.08, 
                                    height_shift_range=0.08,
                                    shear_range=0.10, 
                                    zoom_range=0.08,
                                    channel_shift_range = 10, 
                                    horizontal_flip=True,
                                    fill_mode="constant") 
# datagen_dev = ImageDataGenerator(preprocessing_function = preprocess_input,
#                                     rotation_range=10, 
#                                     width_shift_range=0.1, 
#                                     height_shift_range=0.1,
#                                     shear_range=0.15, 
#                                     zoom_range=0.1,
#                                     channel_shift_range = 10, 
#                                     horizontal_flip=True,
#                                     fill_mode="nearest")                                 
# datagen_test = ImageDataGenerator(preprocessing_function = preprocess_input,
#                                     rotation_range=10, 
#                                     width_shift_range=0.1, 
#                                     height_shift_range=0.1,
#                                     shear_range=0.15, 
#                                     zoom_range=0.1,
#                                     channel_shift_range = 10, 
#                                     horizontal_flip=True,
#                                     fill_mode="nearest")                                   

print("done")

In [ ]:

#####################################
##NO NEED TO RUN EVERY SINGLE TIME ##
#####################################
#To send input images through the frozen part of the network and cache the results, which would be
#used as input for training. This would improve training speed significantly 
vgg = InceptionV3(include_top=False, weights="imagenet",input_shape=(image_size_w, image_size_h, 3))

# #To pop last few layers
# vgg.layers.pop()
# vgg.layers.pop()
# vgg.layers.pop()
# vgg.layers.pop()
# vgg.layers.pop()


##TRAIN SET
#__this can take an hour and half to run so only run it once. 
#once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__
start = datetime.datetime.now()
 
generator_train = datagen_train.flow_from_directory( 
    train_path, 
    target_size=(image_size_w, image_size_h), 
    batch_size=train_batch_size, 
    class_mode=None, 
    shuffle=True) 
 
nb_train_samples = len(generator_train.filenames) 
num_classes = len(generator_train.class_indices) 
 
predict_size_train = int(math.ceil(nb_train_samples / train_batch_size)) 

if includePreTrained == False:
    bottleneck_features_train = vgg.predict_generator(generator_train, predict_size_train, verbose=1) 

    np.save(bottleneck_file_root+"bottleneck_features_train.npy", bottleneck_features_train)
    end= datetime.datetime.now()
    elapsed= end-start
    print ("Time: ", elapsed)
    
# else:
#     prediction_proba1=[]
#     prediction_classes1=[]
#     print("generator_train len: ", len(generator_train))
# #     print("generator_dev: ", len(validation_generator))
# #     print("generator_test: ", len(validation_generator))
#     for i in range(len(generator_train)):
#         print (" array coming...")
#         #print(validation_generator[i])
#         kl = generator_train[i]
# #         print(kl)
#         print("numpy array")
#         print(kl.shape)
#         #print(kl[0])
#         features = vgg.predict_on_batch(kl)
#         print("features")
#         print(features)
#         prediction_proba = vgg.predict_proba(features)
#         prediction_classes = vgg.predict_classes(features)
#         prediction_classes1.extend(prediction_classes)
#         prediction_proba1.extend(prediction_proba)
#         #print(prediction_proba1)
#         print(prediction_classes1)

##DEV SET
start = datetime.datetime.now()

generator_dev = datagen_dev.flow_from_directory( 
    dev_path, 
    target_size=(image_size_w, image_size_h), 
    batch_size=dev_batch_size, 
    class_mode=None, 
    shuffle=True) 
 
nb_dev_samples = len(generator_dev.filenames) 
num_classes = len(generator_dev.class_indices) 
 
predict_size_dev = int(math.ceil(nb_dev_samples / dev_batch_size)) 

if includePreTrained == False:
    bottleneck_features_dev = vgg.predict_generator(generator_dev, predict_size_dev) 

    np.save(bottleneck_file_root+"bottleneck_features_dev.npy", bottleneck_features_dev)
    end = datetime.datetime.now()
    print("Time: ", end-start)

##TEST SET
start = datetime.datetime.now()


generator_test = datagen_test.flow_from_directory( 
    test_path, 
    target_size=(image_size_w, image_size_h), 
    batch_size=test_batch_size, 
    class_mode=None, 
    shuffle=True) 
 
nb_test_samples = len(generator_test.filenames) 
num_classes_ = len(generator_test.class_indices) 
 
predict_size_test = int(math.ceil(nb_test_samples / test_batch_size)) 

if (includePreTrained == False):
    bottleneck_features_test = vgg.predict_generator(generator_test, predict_size_test) 

    np.save(bottleneck_file_root+"bottleneck_features_test.npy", bottleneck_features_test)
    end = datetime.datetime.now()
    print("Time: ", end-start)

# i=0
# for batch in generator_train:
#     i+=1
#     if i > 0: #save 2-1 = one image
#         break  # otherwise the generator would loop indefinitely
# i=0
# for batch in generator_dev:
#     i+=1
#     if i > 0: #save 2-1 = one image
#         break  # otherwise the generator would loop indefinitely
# i=0
# for batch in generator_test:
#     i+=1
#     if i > 0: #save 2-1 = one image
#         break  # otherwise the generator would loop indefinitely
print("done")

In [ ]:
##Loading Training Input Data from cache
#training data
generator_top = datagen_train.flow_from_directory( 
   train_path, 
   target_size=(image_size_w, image_size_h), 
   batch_size=train_batch_size, 
   class_mode="categorical", 
   shuffle=True) 

nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
 
 
# get the class labels for the training data, in the original order 
train_labels = generator_top.classes 
# convert the training labels to categorical vectors 
train_labels = to_categorical(train_labels, num_classes=num_classes)

#class_weight for imbalanced classes
counter = Counter(generator_top.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

print(class_weights)

#validation data
generator_top_dev = datagen_dev.flow_from_directory( 
   dev_path, 
   target_size=(image_size_w, image_size_h), 
   batch_size=dev_batch_size, 
   class_mode="categorical", #none??
   shuffle=True) 

nb_dev_samples = len(generator_top_dev.filenames) 
num_classes_dev = len(generator_top_dev.class_indices) 


 
dev_labels = generator_top_dev.classes 
 
dev_labels = to_categorical(dev_labels, num_classes=num_classes_dev)

#test data
generator_top_test = datagen_test.flow_from_directory( 
   test_path, 
   target_size=(image_size_w, image_size_h), 
   batch_size=test_batch_size, 
   class_mode="categorical", 
   shuffle=True) 
 
nb_test_samples = len(generator_top_test.filenames) 
num_classes_test = len(generator_top_test.class_indices) 

if includePreTrained==False:
    # load the bottleneck features saved earlier 
    train_data = np.load(bottleneck_file_root+"bottleneck_features_train.npy") 
    dev_data = np.load(bottleneck_file_root+"bottleneck_features_dev.npy")  
    test_data = np.load(bottleneck_file_root+"bottleneck_features_test.npy") 
 
test_labels = generator_top_test.classes 
 
test_labels = to_categorical(test_labels, num_classes=num_classes_test)
print("done")

In [ ]:
#print(train_data.shape[1:])
vgg.summary()
for layer in vgg.layers[:5]:
    print(layer)

print("======================================")
for layer in vgg.layers[5:]:
    print(layer)
# for layer in model.layers[249:]:
#     layer.trainable = True

In [ ]:
##Define model and start training
start = datetime.datetime.now()
model = Sequential() 

# #adding back poped 4 conv + 1 pool layer
# print(train_data.shape[1:])
# model.add(ZeroPadding2D((1,1),input_shape=train_data.shape[1:]))
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

trainBool = False

if includePreTrained:
#     # Freeze the layers except the last 4 layers
    # Check the trainable status of the individual layers
    for layer in vgg.layers: ##[:-4]:##
        layer.trainable = True
#         #print(layer, layer.trainable)
#     for layer in vgg.layers[:279]:
#         layer.trainable = False
#     for layer in vgg.layers[280:]:
#         layer.trainable = True

    model.add(vgg)
    
    if modelToLoad == "InceptionV3":             
        #model.add(Flatten())
        model.add(GlobalAveragePooling2D(name='avg_pool'))
        model.add(Dense(1024, activation='relu'
                        ,kernel_regularizer=regularizers.l2(0.005)
                        #activity_regularizer=regularizers.l1(0.005)
                       ))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(0.6)) 
#         model.add(Dense(512, activation='relu'))
#         model.add(LeakyReLU(alpha=0.3))
#         model.add(Dropout(0.3)) 
        model.add(Dense(num_classes, activation='softmax', name='predictions'))
    
    
        for layer in model.layers[0].layers[:-100]:
           layer.trainable = False
        for layer in model.layers[0].layers[-100:]:
           layer.trainable = True

        for layer in model.layers:
            print(layer, ": ", layer.trainable)

    
        model.load_weights("D:/Debian/cs_230_proj/weights/inceptionv3_model_46%_[30].h5")
        
#         for i in range(4):
#             model.pop()

#        model.add(Dense(1024, activation='relu'))
#        model.add(LeakyReLU(alpha=0.3))
#        model.add(Dropout(0.3))
#        model.add(Dense(num_classes, activation='softmax', name='predictions'))

        model.summary()


    
#         model.add(Flatten(input_shape=train_data.shape[1:])) 
#         #FC layers
#         model.add(Dense(256, activation='linear'))
#         model.add(LeakyReLU(alpha=0.3))
#         model.add(Dropout(0.5)) 
#         model.add(Dense(128, activation='linear'))
#         model.add(LeakyReLU(alpha=0.3))
#         model.add(Dropout(0.3)) 
#         model.add(Dense(num_classes, activation="softmax"))
    
    elif modelToLoad == "VGG":
        model.add(Flatten()) 
        #FC layers
        model.add(Dense(256, activation='linear'))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(0.5)) 
        model.add(Dense(128, activation='linear'))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(0.3)) 
        model.add(Dense(num_classes, activation="softmax"))

    #     #https://gist.github.com/didacroyo/839bd1dbb67463df8ba8fb14eb3fde0c
#     # we chose to train the top 2 inception blocks, i.e. we will freeze
#     # the first 249 layers and unfreeze the rest:
#     # in other examples found it was 172 insted 249. 
#     # I took 249 according to https://keras.io/applications/#inceptionv3
#     for layer in model.layers[:280]:
#         layer.trainable = False
#     for layer in model.layers[280:]:
#         layer.trainable = True

    
    
    
else:
    model.add(Flatten(input_shape=train_data.shape[1:])) 
    #FC layers
    model.add(Dense(256, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.5)) 
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.3)) 
    model.add(Dense(num_classes, activation="softmax"))


for layer in vgg.layers:
    print(layer, layer.trainable)
model.summary()


model.compile(loss="categorical_crossentropy",
    #optimizer=optimizers.Adam(lr=2e-4),#optimizers.RMSprop(lr=1e-4),
    optimizer=optimizers.SGD(momentum=0.8, lr=1e-5),#decay=1e-2/epochs
    #optimizer=optimizers.RMSprop(),
    metrics=["acc"])
print("debug " , predict_size_train)
if trainBool:
    if includePreTrained:
        history = model.fit_generator(generator_top, 
                                      steps_per_epoch=predict_size_train,
                                      validation_data=generator_top_dev, 
                                      validation_steps=predict_size_dev, 
                                      epochs = epochs, 
                                      class_weight = class_weights,
                                      verbose = 1,
                                      #use_multiprocessing=True,
                                      max_queue_size=6,
                                      workers=6
    #                                   callbacks=[EarlyStopping(monitor='val_acc', 
    #                                                            min_delta=0.0001, 
    #                                                            patience=5, 
    #                                                            verbose=1, 
    #                                                            mode='auto', 
    #                                                            baseline=None, 
    #                                                            restore_best_weights=True
    #                                                           )]
                                     )

        model.save_weights(top_model_weights_path)

        (eval_loss, eval_accuracy) = model.evaluate_generator(generator_top_dev, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

    else:
        #To use npy input
        history = model.fit(train_data, train_labels, 
           epochs=epochs,
           batch_size=train_batch_size, 
           validation_data=(dev_data, dev_labels))

        model.save_weights(top_model_weights_path)

        (eval_loss, eval_accuracy) = model.evaluate( 
            dev_data, dev_labels, batch_size=dev_batch_size,verbose=1)

    print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100)) 
    print("[INFO] Loss: {}".format(eval_loss)) 

    end= datetime.datetime.now()
    elapsed= end-start
    print ("Time: ", elapsed)



In [ ]:
for layer in vgg.layers:
        print(layer, layer.trainable)
model.summary()


In [ ]:
#Graphing training and validation results
def plotTraining(history):
    acc = history.history["acc"]
    val_acc = history.history["val_acc"]
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(acc))
    plt.plot(epochs, acc, "r", label="Training acc")
    plt.plot(epochs, val_acc, "b", label="Validation acc")
    plt.title("Training and validation accuracy")
    plt.ylabel("accuracy") 
    plt.xlabel("epoch")
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, "r", label="Training loss")
    plt.plot(epochs, val_loss, "b", label="Validation loss")
    plt.title("Training and validation loss")
    plt.ylabel("loss") 
    plt.xlabel("epoch")
    plt.legend()
    plt.show()


plotTraining(history)

##Evaluate on test set
print(model.metrics_names)
if includePreTrained:
#     print(model.evaluate_generator(generator_top_test, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1))
    print(model.evaluate_generator(generator_top_test, steps=predict_size_test, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1))

    #predict_generator(generator_top_test, steps=predict_size_test,verbose=1)
# else:
#     model.evaluate(test_data,test_labels)



In [ ]:
##TOP K accuracy###
import functools
from keras import metrics as M
top3_acc = functools.partial(M.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'
model.compile(loss="categorical_crossentropy",
    optimizer= optimizers.SGD(momentum=0.8, lr=1e-4, decay=1e-2/epochs),
    #optimizers.Adam(lr=2e-4),#optimizers.RMSprop(lr=1e-4),
    #optimizer=optimizers.SGD(momentum=0.8, lr=1e-3, decay=1e-2/epochs),#decay=1e-2/epochs
    #optimizer=optimizers.RMSprop(),
    metrics=["acc",'top_k_categorical_accuracy',top3_acc])
model.load_weights("D:\Debian\cs_230_proj\weights\inceptionv3_model_46%_[30].h5")
print(model.metrics_names)
model.evaluate_generator(generator_top_test, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

In [ ]:
print(preds.shape)

In [ ]:
#Draw Confusion Matrix:
def plot_confusion_matrix(cm1, classes,
   normalize=False,
   title="Confusion matrix",
   cmap=plt.cm.Blues):
 
#Add Normalization Option
#  prints pretty confusion metric with normalization option 
   if normalize:
     cm = cm1.astype('float') / cm1.sum(axis=1)[:, np.newaxis]
     print("Normalized confusion matrix")
   else:
     print("Confusion matrix, without normalization")
 
# print(cm)
 
   plt.imshow(cm, interpolation='nearest', cmap=cmap)
   plt.title(title)
   plt.colorbar()
   tick_marks = np.arange(len(classes))
   plt.xticks(tick_marks, classes, rotation=45)
   plt.yticks(tick_marks, classes)
 
   fmt = ".2f" if normalize else "d"
   thresh = cm.max() / 2.
   for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
 
   plt.tight_layout()
   plt.ylabel("True label")
   plt.xlabel("Predicted label") 

=== To perform error analysis by examing the correctly or incorrectly labeled images

In [ ]:
#funtion to display correctly or incorrectly predicted pictures
def display_images(set,path):
    # Show the set
    for i in range(len(set)):
        pred_class = np.argmax(predictions[set[i]])
        pred_label = idx2label[pred_class]

        title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
            fnames[set[i]].split('/')[0],
            pred_label,
            predictions[set[i]][pred_class])

        original = load_img('{}/{}'.format(path,fnames[set[i]]))
        plt.figure(figsize=[7,7])
        plt.axis('off')
        plt.title(title)
        plt.imshow(original)
        plt.show()

In [ ]:
# #Combine frozen layers and trained layers to form the final full model 
num_classes = 605
vgg = InceptionV3(include_top=False, weights="imagenet",input_shape=(image_size_w, image_size_h, 3))

full_model = Sequential()
full_model.add(vgg)
full_model.add(GlobalAveragePooling2D(name='avg_pool'))
full_model.add(Dense(1024, activation='relu'
                ,kernel_regularizer=regularizers.l2(0.005)
                #activity_regularizer=regularizers.l1(0.005)
               ))
full_model.add(LeakyReLU(alpha=0.3))
full_model.add(Dropout(0.6)) 
#         model.add(Dense(512, activation='relu'))
#         model.add(LeakyReLU(alpha=0.3))
#         model.add(Dropout(0.3)) 
full_model.add(Dense(num_classes, activation='softmax', name='predictions'))

full_model.load_weights("D:/Debian/cs_230_proj/weights/inceptionv3_model_45%_[29].h5")

full_model.compile(loss="categorical_crossentropy",
    #optimizer=optimizers.Adam(lr=2e-4),#optimizers.RMSprop(lr=1e-4),
    optimizer=optimizers.SGD(momentum=0.8, lr=1e-4, decay=1e-2/epochs),#decay=1e-2/epochs
    #optimizer=optimizers.RMSprop(),
    metrics=["acc"])
# print(full_model.summary())

# Create a generator for prediction
datagen_test = ImageDataGenerator(preprocessing_function = preprocess_input)
generator_top_test = datagen_test.flow_from_directory( 
   test_path, 
   target_size=(image_size_w, image_size_h), 
   batch_size=test_batch_size, 
   class_mode="categorical", 
   shuffle=False) 

# print(full_model.evaluate_generator(generator_top_test, steps=predict_size_test, verbose=1))


# Get the filenames from the generator
fnames = generator_top_test.filenames
 
# Get the ground truth from generator
ground_truth  = generator_top_test.classes
 
# Get the label to class mapping from the generator
label2index = generator_top_test.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())   


# # # Get the predictions from the model using the generator
generator_top_test.reset()
predictions = full_model.predict_generator(generator_top_test, steps=predict_size_test,verbose=1)
# # classes = generator_top_test.classes[generator_top_test.index_array]
predicted_classes  = np.argmax(predictions, axis=-1)
# # print(sum(predicted_classes==classes)/11622)

# # to get one batch of images and labels
# y_img_batch, y_class_batch = generator_top_test[1] 
# y_pred = np.argmax(full_model.predict(y_img_batch),-1)
# y_true = np.argmax(y_class_batch,-1)
# print(sum(y_pred==y_true)/test_batch_size)
# print(test_batch_size)




# y_pred = full_model.predict_generator(generator_top_test, steps=predict_size_test,verbose=1)
# y_true = np.argmax(predictions,axis=1)    
 
errors = np.where(predicted_classes != ground_truth )[0]
corrects = np.where(predicted_classes  == ground_truth )[0]

print("No of errors = {}/{}".format(len(errors),generator_top_test.samples)) 
print("No of corrects = {}/{}".format(len(corrects),generator_top_test.samples))
# print("====Errors=====")
# display_images(errors,test_path)
# print("====Corrects=====")
# display_images(corrects,test_path)



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
generator_top_test.reset()
# #model.evaluate(test_data,test_labels)
# Y_pred = model.predict_generator(generator_top_test, verbose=1)
# #preds = np.round(model.predict(test_data),0)
# preds = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
#print(confusion_matrix(np.argmax(test_labels,axis=1).tolist(), np.argmax(preds, axis=1).tolist()))
print(confusion_matrix(generator_top_test.classes, predicted_classes))
print('Classification Report')
target_names = os.listdir(train_path)
print(classification_report(generator_top_test.classes, predicted_classes, target_names=target_names))


# print('rounded test_labels',preds)
# classesList = os.listdir(train_path)
# classification_metrics = metrics.classification_report(test_labels,preds,target_names=classesList)
# print(classification_metrics)

#cm2 = confusion_matrix(np.argmax(test_labels,axis=1).tolist(), predicted_classes.tolist())
cm2 = confusion_matrix(generator_top_test.classes[:10], predicted_classes[:10])
plot_confusion_matrix(cm2,
                     generator_top_test.classes[:10],
                     normalize=True
                    )

In [ ]:
# Show the errors
array = errors
error_makemodel = 0
for i in range(len(array)):
    r = np.random.randint(0,len(array))
    #r = i
    pred_class = np.argmax(predictions[array[r]])
    pred_label = idx2label[pred_class]
    
    actual_makemodel = fnames[array[r]].split('/')[0]
    actual_makemodel = actual_makemodel.split('_')[0] + "_" + actual_makemodel.split('_')[1]
    
    pred_makemodel = pred_label.split('_')[0] + "_" + pred_label.split('_')[1]
    if pred_makemodel != actual_makemodel:
        print("pred: ", pred_makemodel, "actual: ", actual_makemodel)
        error_makemodel += 1
        title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
            fnames[array[r]].split('/')[0],
            pred_label,
            predictions[array[r]][pred_class])

        original = load_img('{}/{}'.format(test_path,fnames[array[r]]))
        plt.figure(figsize=[7,7])
        plt.axis('off')
        plt.title(title)
        plt.imshow(original)
        plt.show()
print(error_makemodel)

In [ ]:
##Test to load and modify models from .h5
loadModel = Sequential()

loadModel.add(vgg)

#model.add(Flatten())
loadModel.add(GlobalAveragePooling2D(name='avg_pool'))
loadModel.add(Dense(1024, activation='relu'
                ,kernel_regularizer=regularizers.l2(0.005)
                #activity_regularizer=regularizers.l1(0.005)
               ))
loadModel.add(LeakyReLU(alpha=0.3))
loadModel.add(Dropout(0.5)) 
loadModel.add(Dense(512, activation='relu'))
loadModel.add(LeakyReLU(alpha=0.3))
loadModel.add(Dropout(0.3)) 
loadModel.add(Dense(num_classes, activation='softmax', name='predictions'))

loadModel.load_weights("D:\Debian\cs_230_proj\inceptionv3_model_top40-66%_[16-4].h5")
loadModel.summary()

# for i in range(4):
#     loadModel.pop()

# loadModel.add(Dense(1024, activation='relu'))
# loadModel.add(LeakyReLU(alpha=0.3))
# loadModel.add(Dropout(0.3)) 
# loadModel.add(Dense(604, activation='softmax', name='predictions'))

# loadModel.summary()

# for layer in loadModel.layers[:-8]:
#     layer.trainable = False
# for layer in loadModel.layers[-7:]:
#     layer.trainable = True
    
# for layer in loadModel.layers:
#     print(layer, ": ", layer.trainable)

# for layers in loadModel.layers[0].layers:
#     print(layers)
# print(loadModel.layers[0])    


In [ ]:
#To visualize Filters and Features
#https://towardsdatascience.com/visualising-filters-and-feature-maps-for-deep-learning-d814e13bd671
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model
import matplotlib.pyplot as plt
import numpy as np

full_model = Sequential()
full_model.add(vgg)
full_model.add(GlobalAveragePooling2D(name='avg_pool'))
full_model.add(Dense(1024, activation='relu'
                ,kernel_regularizer=regularizers.l2(0.005)
                #activity_regularizer=regularizers.l1(0.005)
               ))
full_model.add(LeakyReLU(alpha=0.3))
full_model.add(Dropout(0.6)) 
#         model.add(Dense(512, activation='relu'))
#         model.add(LeakyReLU(alpha=0.3))
#         model.add(Dropout(0.3)) 
full_model.add(Dense(num_classes, activation='softmax', name='predictions'))

full_model.load_weights("D:/Debian/cs_230_proj/weights/inceptionv3_model_45%_[29].h5")

full_model.compile(loss="categorical_crossentropy",
    #optimizer=optimizers.Adam(lr=2e-4),#optimizers.RMSprop(lr=1e-4),
    optimizer=optimizers.SGD(momentum=0.8, lr=1e-4, decay=1e-2/epochs),#decay=1e-2/epochs
    #optimizer=optimizers.RMSprop(),
    metrics=["acc"])
full_model.summary()
layer_dict = dict([(layer.name, layer) for layer in full_model.layers])

layer_name = 'inception_v3'
# for _ in range(4):
#     full_model.pop()

full_model.summary()
# model = Model(inputs=full_model.inputs, output = layer_dict.get_output_at(0)) #outputs=layer_dict[layer_name].output)

# Perpare the image
image = load_img('D:/Debian/cs_230_proj/VMMRdb_year2000+_sample100+/test/acura_mdx_2001/2001 acura mdx_01212_5kttdWUTmZQ_600x450.jpg', target_size=(image_size_w, image_size_h))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = preprocess_input(image)

# Apply the model to the image
feature_maps = full_model.predict(image)

square = 8
index = 1
for _ in range(square):
	for _ in range(square):
        
		ax = plt.subplot(square, square, index)
		ax.set_xticks([])
		ax.set_yticks([])

		plt.imshow(feature_maps[0, :, :, index-1], cmap='viridis')
		index += 1
        
plt.show()

In [ ]:
##From: https://github.com/molohov/cs230-project/blob/master/dev/visualize_nn_layer.py##
from keras import backend as K
from vis.losses import ActivationMaximization
from vis.regularizers import TotalVariation, LPNorm
import numpy as np
from vis.visualization import visualize_activation
from vis.visualization import get_num_filters
from vis.utils import utils
from keras import activations

from matplotlib import pyplot as plt
#from scipy.misc import imresize

from PIL import Image
from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Input, BatchNormalization, Conv2D
from vis.input_modifiers import Jitter


## you must install this version of keras-vis from the repo in order for this to work
## pip install git+https://github.com/raghakot/keras-vis.git --upgrade

## otherwise you'll get this weird error
## tensorflow.python.framework.errors_impl.InvalidArgumentError: input_1:0 is both fed and fetched.



# load model
model = full_model.layers[0]
#model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(150, 150, 3)))
print('Model loaded.')
#model.summary()
single_filter = False
#single_filter = True

model.summary()
# The name of the layer we want to visualize
layer_name = 'conv2d_1'
layer_idx = utils.find_layer_idx(model, layer_name)

# Visualize all filters in this layer.
filters = None
if (single_filter == False):
    filters = np.arange(get_num_filters(model.layers[layer_idx]))
else:
    filters = ([0])

# Generate input image for each filter.
vis_images = []
print ("Processing ", len(filters), " filters")
for idx in filters:
    print ("Processing filter", idx, "/", len(filters))
    img = visualize_activation(model, layer_idx, filter_indices=idx, tv_weight=0., max_iter=100, input_modifiers=[Jitter(0.05)])

    # Utility to overlay text on image.
    img = utils.draw_text(img, 'Filter {}'.format(idx))    
    vis_images.append(img)

# Generate stitched image palette with 8 cols.
stitched = utils.stitch_images(vis_images, cols=8)    
plt.axis('off')
plt.title(layer_name)
plt.imsave(layer_name + ".png", stitched)
#plt.imshow(stitched)
#plt.show()
#stitched = Image.fromarray(stitched)
#stitched.save(layer_name + '.png')

In [ ]:
## Convert Sequential model to function
input_layer = layers.Input(batch_shape=full_model.layers[0].input_shape)
prev_layer = input_layer
for layer in full_model.layers:
    layer._inbound_nodes = []
    prev_layer = layer(prev_layer)

funcmodel = models.Model([input_layer], [prev_layer])
funcmodel.summary()

In [ ]:
##https://edebrouwer.github.io/deeplearning/carvision/visualization/neural/networks/learning/2017/08/09/Deep_Visualization.html
from vis.visualization import visualize_activation,visualize_saliency,overlay,visualize_cam
from vis.utils import utils
from keras import activations
from keras import models

full_model = Sequential()
base_model = InceptionV3(weights='imagenet',include_top=False,input_shape = (299, 299, 3))

full_model.add(base_model)
full_model.add(GlobalAveragePooling2D(name='avg_pool'))
full_model.add(Dense(1024, activation='relu'
                ,kernel_regularizer=regularizers.l2(0.005)
                #activity_regularizer=regularizers.l1(0.005)
               ))
full_model.add(LeakyReLU(alpha=0.3))
full_model.add(Dropout(0.6)) 
#         model.add(Dense(512, activation='relu'))
#         model.add(LeakyReLU(alpha=0.3))
#         model.add(Dropout(0.3)) 
full_model.add(Dense(num_classes, activation='softmax', name='predictions'))

full_model.load_weights("D:/Debian/cs_230_proj/weights/inceptionv3_model_45%_[29].h5")

# full_model.layers[0].summary()

new_model = Sequential()
for layer in full_model.layers[0].layers:
    new_model.add(layer)
    print(layers)


print(len(full_model.get_weights()))
# full_model.compile(loss="categorical_crossentropy",
#     #optimizer=optimizers.Adam(lr=2e-4),#optimizers.RMSprop(lr=1e-4),
#     optimizer=optimizers.SGD(momentum=0.8, lr=1e-4, decay=1e-2/epochs),#decay=1e-2/epochs
#     #optimizer=optimizers.RMSprop(),
#     metrics=["acc"])

# base_model = InceptionV3(weights='imagenet',include_top=False,input_shape = (299, 299, 3))
# x = base_model.output
# x = GlobalAveragePooling2D(name='avg_pool')(x)

# #Only for version 2:
# x = Dense(1024, activation='relu'
#                 ,kernel_regularizer=regularizers.l2(0.005))(x)
# x = LeakyReLU(alpha=0.3)(x)
# x = Dropout(0.6)(x)
# predictions = Dense(num_classes, activation='softmax',name='predictions')(x)

# func_model= Model(inputs=base_model.input, outputs=predictions)
# print("layer counter: ",len(func_model.layers))
# func_model.summary()
# func_model.load_weights("D:/Debian/cs_230_proj/weights/inceptionv3_model_45%_[29].h5", by_name=True)
# #model_origin = InceptionV3(weights='imagenet', include_top=True)
# func_model.compile(loss="categorical_crossentropy",
#     #optimizer=optimizers.Adam(lr=2e-4),#optimizers.RMSprop(lr=1e-4),
#     optimizer=optimizers.SGD(momentum=0.8, lr=1e-4, decay=1e-2/epochs),#decay=1e-2/epochs
#     #optimizer=optimizers.RMSprop(),
#     metrics=["acc"])
# print(func_model.evaluate_generator(generator_top_test, steps=predict_size_test, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1))



# Utility to search for layer index by name.
# Alternatively we can specify this as -1 since it corresponds to the last layer.
layer_idx = utils.find_layer_idx(full_model, 'predictions')
print("Remove Activation from Last Layer")
# Swap softmax with linear
full_model.layers[layer_idx].activation = activations.linear
print("Done. Now Applying changes to the model ...")
full_model = utils.apply_modifications(full_model)
im_files=["D:/Debian/cs_230_proj/VMMRdb_year2000+_sample100+/test/chevrolet_silverado_2014/2014 chevrolet 1500 silverado_00J0J_aNCzTaQOp4n_600x450.jpg"]
for im_file in im_files:
    img1 = image.load_img(im_file,target_size=(299,299))
    img1 = image.img_to_array(img1)
    img1 = np.expand_dims(img1, axis=0)
    img1 = preprocess_input(img1)
    layer_idx = utils.find_layer_idx(full_model, 'predictions')
    heatmap = visualize_cam(full_model, layer_idx, filter_indices=1, seed_input=img1[0,:,:,:])
    img_init=utils.load_img(im_file,target_size=(299,299))
    plt.imshow(overlay(img_init, heatmap))
    plt.show()

In [ ]:
###https://machinelearningmastery.com/how-to-visualize-filters-and-feature-maps-in-convolutional-neural-networks/
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot

vgg = InceptionV3(include_top=False, weights="imagenet",input_shape=(image_size_w, image_size_h, 3))


full_model = Sequential()
full_model.add(vgg)
full_model.add(GlobalAveragePooling2D(name='avg_pool'))
full_model.add(Dense(1024, activation='relu'
                ,kernel_regularizer=regularizers.l2(0.005)
                #activity_regularizer=regularizers.l1(0.005)
               ))
full_model.add(LeakyReLU(alpha=0.3))
full_model.add(Dropout(0.6)) 
#         model.add(Dense(512, activation='relu'))
#         model.add(LeakyReLU(alpha=0.3))
#         model.add(Dropout(0.3)) 
full_model.add(Dense(605, activation='softmax', name='predictions'))

full_model.load_weights("D:/Debian/cs_230_proj/weights/inceptionv3_model_45%_[29].h5")
####################
### Plot filters ###
###################
# for layer in full_model.layers[0].layers:
#     # check for convolutional layer
#     if 'conv' not in layer.name:
#         continue
#     # get filter weights
#     filters = layer.get_weights()[0]
#     print(layer.name, filters.shape)
  
#     # normalize filter values to 0-1 so we can visualize them
#     f_min, f_max = filters.min(), filters.max()
#     filters = (filters - f_min) / (f_max - f_min)

#     # plot first few filters
#     n_filters, ix = 6, 1
#     for i in range(n_filters):
#         # get the filter
#         f = filters[:, :, :, i]
#         # plot each channel separately
#         for j in range(3):
#             # specify subplot and turn of axis
#             ax = pyplot.subplot(n_filters, 3, ix)
#             ax.set_xticks([])
#             ax.set_yticks([])
#             # plot filter channel in grayscale
#             pyplot.imshow(f[:, :, j], cmap='gray')
#             ix += 1
#     # show the figure
#     pyplot.show()

#####################
## Plot Feature Map##
#####################
# summarize feature map shapes
conv_idx_array = []
inv3_model = full_model.layers[0]
for i in range(len(inv3_model.layers)):
    layer = inv3_model.layers[i]
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    # summarize output shape
    conv_idx_array.append(i)
    print(i, layer.name, layer.output.shape, layer.output.shape[3])
print("sum of output = ", np.sum(conv_idx_array))
# load the image with the required shape
img = load_img('D:/Debian/cs_230_proj/VMMRdb/volkswagen_gti_2015/2015 volkswagen gti_01111_jBKFWm6SB9U_600x450.jpg', target_size=(299, 299))
plt.imshow(img)
plt.show()
# convert the image to an array
img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
img = np.expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
img = preprocess_input(img)

for convIdx in conv_idx_array:
    # redefine model to output right after the first hidden layer
    model = Model(inputs=inv3_model.inputs, outputs=inv3_model.layers[convIdx].output)
    # get feature map for first hidden layer
    feature_maps = model.predict(img)
    # plot all 64 maps in an 8x8 squares
    square = 8
    ix = 1
    layerName = inv3_model.layers[convIdx].name
    #layerName = layerName.split("_")[0] + "_" + str(int(layerName.split("_")[1]) - 84)
    print(layerName)
    pyplot.figure(figsize = (20,20))
    for _ in range(square):
        for _ in range(square):
            if ix <= inv3_model.layers[convIdx].output.shape[3]:
                # specify subplot and turn of axis
                ax = pyplot.subplot(square, square, ix)
                ax.set_xticks([])
                ax.set_yticks([])
                # plot filter channel in grayscale
                pyplot.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
                ix += 1
    # show the figure
    pyplot.show()

=====Below code is for references only=====

In [ ]:

# classesList = os.listdir(train_path)
# print(classesList)
# class_size = len(classesList)
# # datagen = ImageDataGenerator(rescale=1./255)
# # generator = datagen.flow_from_directory(
# #     train_path,
# #     target_size=(image_size_h, image_size_w), 
# #     #classes=classesList, 
# #     class_mode=None,
# #     batch_size=train_batch_size
# #     #color_mode='grayscale'
# # )



# train_batches = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     vertical_flip=True,
#     fill_mode='nearest'
# ).flow_from_directory(
#     train_path, 
#     target_size=(image_size_h, image_size_w), 
#     classes=classesList, 
#     batch_size=train_batch_size
#     #color_mode='grayscale'
# )

# dev_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
#     dev_path, 
#     target_size=(image_size_h, image_size_w), 
#     classes=classesList, 
#     batch_size=dev_batch_size
#     #color_mode='grayscale'
# #     class_mode=None
# )

# test_batches = ImageDataGenerator().flow_from_directory(
#     test_path, 
#     target_size=(image_size_h, image_size_w), 
#     #classes=classesList, 
#     batch_size=test_batch_size
#    # color_mode='grayscale'
# )




In [ ]:
# #source: https://github.com/smileservices/keras_utils/blob/master/utils.py
# # plots images with labels within jupyter notebook
# def plots(ims, figsize=(60,30), rows=2, interp=False, titles=None):
#     if type(ims[0]) is np.ndarray:
#         ims = np.array(ims).astype(np.uint8)
#         if (ims.shape[-1] != 3):
#             ims = ims.transpose((0,2,3,1))
#     f = plt.figure(figsize=figsize)
#     cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
#     for i in range(len(ims)):
#         sp = f.add_subplot(rows, cols, i+1)
#         sp.axis('Off')
#         if titles is not None:
#             sp.set_title(titles[i], fontsize=16)
#         plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
# imgs,labels= next(train_batches)

In [ ]:
# print(imgs.shape)
# plots(imgs,titles=labels)

In [ ]:
# #Load the VGG model
# vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size_h, image_size_w, 3))

# for layer in vgg_conv.layers[:-4]:
#     layer.trainable = False
# for layer in vgg_conv.layers:
#     print(layer)

# # model =Sequential([
# #     Conv2D(32,(3,3),activation='relu',input_shape=(382,512,3)),
# #     Flatten(),
# #     Dense(8,activation='softmax'),
# # ])
# # model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

# # model.fit_generator(train_batches, steps_per_epoch=34,validation_data=dev_batches, validation_steps=4, epochs = 5, verbose = 2)


In [ ]:
# #define the model
# model= Sequential([
#     vgg_conv,
#     Flatten(),
#     Dense(512,activation='relu'),
#     Dropout(0.8),
#     Dense(class_size, activation='softmax')
# ])
# model.summary()

# optimizer = optimizers.Adam()#lr=0.001)


==References Code End===